In [61]:
# import libraries

from bs4 import BeautifulSoup
import requests
import time
import datetime

import smtplib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline

In [62]:
URL = 'https://www.amazon.com/Funny-Data-Systems-Business-Analyst/dp/B07FNW9FGJ/ref=sr_1_3?dchild=1&keywords=data%2Banalyst%2Btshirt&qid=1626655184&sr=8-3&customId=B0752XJYNL&th=1'

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/132.0.0.0 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
page = requests.get(URL, headers=headers)
response = requests.get(URL)


In [63]:
soup = BeautifulSoup(response.text, 'lxml')

#Utilising css selectors
#here I utilized lxml to parse

title_element = soup.select_one("#productTitle")
title = title_element.text.strip()

rating_element = soup.select_one("#acrPopover")
rating = rating_element.get("title")
rating = rating.replace('out of 5 stars', '')

price_element = soup.select_one('span.a-price').select_one('span.a-offscreen')
price = price_element.text.strip()

image_element = soup.select_one("#imgTagWrapperId img")
image_url = image_element.get("src")

print(title)
print(rating)
print(price)
print(image_url)

Got Data Funny Business Data Analyst T-Shirt
4.4 
$19.99
https://m.media-amazon.com/images/I/B1pppR4gVKL._CLa%7C2140%2C2000%7C41l4vVJMxEL.png%7C0%2C0%2C2140%2C2000%2B0.0%2C0.0%2C2140.0%2C2000.0_AC_SX342_SY445_.png


In [64]:
#Here I utilised html parser to parse
soup = BeautifulSoup(response.text, 'html.parser')

title_element = soup.find(id="productTitle")
title = title_element.get_text()
title = title.strip()

rating_element = soup.find(id="acrPopover")
rating = rating_element.get("title")
rating = rating.replace('out of 5 stars', '')

price_element = soup.find('span', class_='a-price')
price_element = price_element.find('span', class_='a-offscreen')
price = price_element.get_text()
price = price.strip()

print(title)
print(rating)
print(price)


Got Data Funny Business Data Analyst T-Shirt
4.4 
$19.99


In [65]:
def parse_listing(listing_url):
    response = requests.get(listing_url, headers=headers)
    soup_search= BeautifulSoup(response.text, "lxml")
    link_elements = soup_search.select("[data-asin] h2 a")
    page_data = []
    for link in link_elements:
        full_url = urljoin(listing_url, link.attrs.get("href"))
        product_info = get_product_info(full_url)
        page_data.append(product_info)

    next_page_el = soup_search.select_one('a:contains("Next")')
    if next_page_el:
        next_page_url = urljoin(listing_url, next_page_el.attrs.get("href"))
        page_data += parse_listing(next_page_url)

    return page_data

    df = pd.DataFrame(page_data)
    df.head()

    print(df)

In [66]:
import csv

# ... (your existing code) ...

# Create a list of dictionaries to store the data
data = [{'Title': title, 'Rating': rating, 'Price': price, 'Image URL': image_url}]

# Specify the CSV file name
csv_file = 'amazon_product_data.csv'

# Write the data to the CSV file
with open(csv_file, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=data[0].keys())
    writer.writeheader()  # Write the header row
    writer.writerows(data)  # Write the data rows

print(f"Data exported to {csv_file}")

Data exported to amazon_product_data.csv


In [67]:
#As an excel file

df = pd.DataFrame(data)

# Specify the Excel file name
excel_file = 'amazon_product_data.xlsx'

# Export the DataFrame to Excel
df.to_excel(excel_file, index=False)  # Set index=False to avoid writing row numbers

print(f"Data exported to {excel_file}")

Data exported to amazon_product_data.xlsx
